<a href="https://colab.research.google.com/github/kumar6rishabh/UNET_from_scratch/blob/main/UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#

In [ ]:
import torch
import torch.nn as nn

In [ ]:
def double_conv(in_channels , out_channels):
  block = nn.Sequential(
      nn.Conv2d(in_channels , out_channels , kernel_size = 3 , stride = 1 , padding = 0) , 
      nn.Conv2d(out_channels , out_channels , kernel_size = 3 , stride = 1 , padding = 0)
  )
  return block
def double_back_conv(in_channels , out_channels):
  block = nn.Sequential(
      nn.Conv2d(in_channels , out_channels , kernel_size = 3 , stride = 1 , padding = 0) , 
      nn.Conv2d(out_channels , out_channels , kernel_size = 3 , stride = 1 , padding = 0)
  )
  return block

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net , self).__init__()

    #Expanding path
    self.exp1 = double_conv(3 , 64)
    self.exp2 = double_conv(64 , 128)
    self.exp3 = double_conv(128 , 256)
    self.exp4 = double_conv(256 , 512)
    self.exp5 = double_conv(512 , 1024)
    self.pool = nn.MaxPool2d(kernel_size = 2)

    #Decompressing path
    self.up_conv12 = nn.ConvTranspose2d(1024 , 512 , kernel_size = 2 , stride = 2)
    self.up_conv34 = nn.ConvTranspose2d(512 , 256 , kernel_size = 2 , stride = 2)
    self.up_conv56 = nn.ConvTranspose2d(256 , 128 , kernel_size = 2 , stride = 2)
    self.up_conv78 = nn.ConvTranspose2d(128 , 64 , kernel_size = 2 , stride = 2)
    self.final_conv = nn.Conv2d(64 , 2 , kernel_size = 3 , stride = 1 , padding = 1)

  def forward(self , x):
    x1 = self.exp1(x)
    x2 = self.pool(x1)
    x3 = self.exp2(x2)
    x4 = self.pool(x3)
    x5 = self.exp3(x4)
    x6 = self.pool(x5)
    x7 = self.exp4(x6)
    x8 = self.pool(x7)
    x = self.exp5(x8)

    x = self.up_conv12(x) #1 , 512 , 56 , 56
    x7 = x7[: , : , 4 : 60 , 4 : 60]
    x = torch.cat((x , x7) , dim = 1)
    x = double_back_conv(1024 , 512)(x)
    x = self.up_conv34(x)
    x5 = x5[: , : , 16 : 120 , 16 : 120]
    x = torch.cat((x , x5) , dim = 1)
    x = double_back_conv(512 , 256)(x)
    x = self.up_conv56(x)
    x3 = x3[: , : , 40 : 240 , 40 : 240]
    x = torch.cat((x , x3) , dim = 1)
    x = double_back_conv(256 , 128)(x)
    x = self.up_conv78(x)
    x1 = x1[: , : , 88 : 480 , 88 : 480]
    x = torch.cat((x1 , x) , dim = 1)
    x = double_back_conv(128 , 64)(x)
    x = self.final_conv(x)
    return x

In [ ]:
Net()(torch.randn(5 , 3 , 572 , 572)).shape

torch.Size([5, 2, 388, 388])